# Questioning Barbie and Oppenheimer Through the Use of Agents

In the following notebook we will build an application that queries both the Barbie and Oppenheimer movies Wikipedia pages, as well as their reviews. 

The main focus of this notebook is to showcase a brief introduction to Agents.

## Build 🏗️

There are 3 main tasks in this notebook:

1. Contruct a Barbie retriever
2. Construct an Oppenheimer retriever
3. Combine the two and allow users to query both resources from a single input through the use of Agents

## Ship 🚢

Based on Tuesday's session - construct a Chainlit (or Gradio) application that allows users to interface with the application.

## Share 🚀

Make a social media post about your final application.

### Dependencies

As always, let's start with some dependencies!

In [ ]:
!pip install -q -U langchain openai

In [ ]:
import getpass
import os
openai_api_key = getpass.getpass("Enter your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

### LLM 

We will be leveraging OpenAI's `gpt-3.5-turbo` throughout the notebook, and we can keep it consistent throughout!

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature = 0)

### Data Collection and Transformation

We'll be leveraging the `WikipediaLoader` tool to collect information from Wikipedia. 

Be sure to set the `doc_content_chars_max` parameter so that you capture the *entire* Wikipedia article content.

In [ ]:
!pip install -q -U wikipedia

In [ ]:
from langchain.document_loaders import WikipediaLoader, CSVLoader

barbie_wikipedia_docs = WikipediaLoader(
    query="Barbie (film)", 
    load_max_docs= 1, 
    doc_content_chars_max= 1_000_000 
    ).load()

barbie_csv_docs = CSVLoader(
    file_path="./barbie_data/barbie.csv",
    source_column="Review_Url
    ).load()

Since we'll be using same format source documentation separated by topic, we can save ourselves some extra effort and set up our splitters once. 

We're going to leverage the `RecursiveCharacterTextSplitter` again, this time paying close attention to the format our Wikipedia articles and reviews are in so we can be sure to chunk them appropritately. 

> HINT: You can pass a list of separators when you intialize your `RecursiveTextSplitter`! They are acted on in order of element 0 -> element len(list).

RELEVANT DOCS:
- [`RecursiveCharacterTextSplitter`](https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#langchain.text_splitter.RecursiveCharacterTextSplitter)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

wikipedia_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500, 
    chunk_overlap = 0,
    length_function = len, 
    is_separator_regex= False,
    separators = ["\n==", "\n", " "]
)

csv_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, 
    chunk_overlap = 100, 
    length_function = len, 
    is_separator_regex= False,
    separators = ["\n", " "]
)

chunked_barbie_wikipedia_docs = wikipedia_text_splitter.transform_documents(barbie_wikipedia_docs)
chunked_barbie_csv_docs = csv_text_splitter.transform_documents(barbie_csv_docs)

#### Retrieval and Embedding Strategy

We've already discussed the useful application of `CacheBackedEmbeddings`, so let's do it again!

RELEVANT DOCS:
- [`CacheBackedEmbeddings`](https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.cache.CacheBackedEmbeddings.html#langchain-embeddings-cache-cachebackedembeddings)

In [ ]:
!pip install -q -U rank_bm25 tiktoken faiss-cpu

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

# set up cached embeddings store
store = LocalFileStore("./cache")

core_embeddings_model = OpenAIEmbeddings()

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model,
    store,
    namespace=core_embeddings_model.model
)

We'll implement a `FAISS` vectorstore, and create a retriever from it.

In [101]:
barbie_csv_faiss_retriever = FAISS.from_documents(chunked_barbie_csv_docs, embedder).as_retriever()

There are a number of excellent options to retrieve documents - we'll be looking at an additional example today, which is called the `EnsembleRetriever`.

The method this is using is outlined in [this paper](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf).

The brief explanation is:

1. We collect results from two different retrieval methods over the same corpus
2. We apply a reranking algorithm to rerank our source documents to be the *most relevant* without losing specific or potentially low-ranked information rich documents
3. We feed the top-k results into the LLM with our query as context.

> HINT: Your weight list should be of type `List[float]` and the `sum(List[float])` should be `1`.

In [102]:
# set up BM25 retriever
barbie_wikipedia_bm25_retriever = BM25Retriever.from_documents(
    chunked_barbie_wikipedia_docs
)

barbie_wikipedia_bm25_retriever.k = 1

# set up ensemble retriever
barbie_ensemble_retriever = EnsembleRetriever(
    retrievers=[barbie_wikipedia_bm25_retriever, barbie_csv_faiss_retriever], 
    weights= [0.5, 0.5]
)

#### Retrieval Agent

We can create a simple conversational retrieval Agent by using the built-ins provided by LangChain!

> HINT: Be sure to provide good natural language descriptions of what the tool should be used for to get the best results.

RELEVANT DOCS:
- [`create_retriever_tool`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool.html#langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool)

In [139]:
from langchain.agents.agent_toolkits import create_retriever_tool

barbie_wikipedia_retrieval_tool = create_retriever_tool(
    retriever = barbie_wikipedia_bm25_retriever,
    name = "BarbieWikipedia",
    description= "Useful for looking up facts about the Barbie movie on wikipedia"
)

barbie_csv_retrieval_tool = create_retriever_tool(
    retriever = barbie_csv_bm25_retriever,
    name = "BarbieReviews",
    description = "Useful for looking up reviews about the Barbie movie in csv files"
)

barbie_retriever_tools = [barbie_wikipedia_retrieval_tool, barbie_csv_retrieval_tool]

Now that we've created our tools, we can combined them into an agent!

RELEVANT DOCS:
- [`create_conversational_retrieval_agent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent.html#langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent)

In [140]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

barbie_retriever_agent_executor = create_conversational_retrieval_agent(llm, barbie_retriever_tools, verbose=True)

In [141]:
barbie_retriever_agent_executor({"input" : "Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?"})



> Entering new AgentExecutor chain...

Invoking: `BarbieReviews` with `Barbie`


[Document(page_content=": 39\nReview_Date: 23 July 2023\nAuthor: Anurag-Shetty\nRating: 10\nReview_Title: A wholesome delight!\nReview: Barbie is based on Mattel's iconic plastic dolls of the same name. Barbie(Margot Robbie) lives a very happy life in Barbie Land, along with her fellow Barbies, Ken(Ryan Gosling) & his fellow Kens. Suddenly, Barbie goes through an existential crisis that makes her question Barbie Land & the truth of her existence.\nReview_Url: /review/rw9199947/?ref_=tt_urv", metadata={'source': '/review/rw9199947/?ref_=tt_urv', 'row': 39})]According to a review I found, the movie Barbie received a rating of 10 and was described as a "wholesome delight". The reviewer mentioned that the movie is about Barbie, who lives a happy life in Barbie Land with her fellow Barbies and Kens. However, Barbie goes through an existential crisis that makes her question Barbie Land and the truth of her exi

{'input': 'Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?',
 'chat_history': [HumanMessage(content='Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'BarbieReviews', 'arguments': '{\n  "__arg1": "Barbie"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content=": 39\\nReview_Date: 23 July 2023\\nAuthor: Anurag-Shetty\\nRating: 10\\nReview_Title: A wholesome delight!\\nReview: Barbie is based on Mattel\'s iconic plastic dolls of the same name. Barbie(Margot Robbie) lives a very happy life in Barbie Land, along with her fellow Barbies, Ken(Ryan Gosling) & his fellow Kens. Suddenly, Barbie goes through an existential crisis that makes her question Barbie Land & the truth of her existence.\\nReview_Url: /review/rw

In [142]:
barbie_retriever_agent_executor({"input" : "What is a very quick summary of the plot of the Barbie movie?"})



> Entering new AgentExecutor chain...

Invoking: `BarbieWikipedia` with `Barbie`


[Document(page_content="Arriving at Venice Beach, Barbie punches a man for groping her, leading to her and Ken's brief arrest. Alarmed by their presence, Mattel's CEO orders their recapture. Barbie tracks down her owner, a tween girl named Sasha, who criticizes her for encouraging unrealistic beauty standards. Distraught, Barbie discovers that Gloria, a Mattel employee and Sasha's mother, inadvertently catalyzed her existential crisis after Gloria began playing with Sasha's old Barbie toys in a similar state. Mattel", metadata={'title': 'Barbie (film)', 'summary': 'Barbie is a 2023 American fantasy comedy film directed by Greta Gerwig from a screenplay she wrote with Noah Baumbach. Based on the Barbie fashion dolls by Mattel, it is the first live-action Barbie film after numerous computer-animated films and specials. The film follows Barbie (Margot Robbie) and Ken (Ryan Gosling) on a journey of self-di

{'input': 'What is a very quick summary of the plot of the Barbie movie?',
 'chat_history': [HumanMessage(content='Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'BarbieReviews', 'arguments': '{\n  "__arg1": "Barbie"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content=": 39\\nReview_Date: 23 July 2023\\nAuthor: Anurag-Shetty\\nRating: 10\\nReview_Title: A wholesome delight!\\nReview: Barbie is based on Mattel\'s iconic plastic dolls of the same name. Barbie(Margot Robbie) lives a very happy life in Barbie Land, along with her fellow Barbies, Ken(Ryan Gosling) & his fellow Kens. Suddenly, Barbie goes through an existential crisis that makes her question Barbie Land & the truth of her existence.\\nReview_Url: /review/rw9199947/?ref_=tt_urv", metadata={\'source\': \'/review/rw9199947

### Oppenheimer Retrieval System

We're going to repourpose some of what we created previously, but this time we'll explore a different multi-source retrieval system.

In [108]:
oppenheimer_wikipedia_docs = WikipediaLoader(
    query = "Oppenheimer (film)",
    load_max_docs = 1,
    doc_content_chars_max = 1_000_000,
).load()

oppenheimer_csv_docs = CSVLoader(
    file_path = "./oppenheimer_data/oppenheimer.csv",
    source_column="Review_Url"
).load()

In [120]:
#chunk up docs
chunked_opp_wikipedia_docs = wikipedia_text_splitter.transform_documents(oppenheimer_wikipedia_docs)
chunked_opp_csv_docss = csv_text_splitter.transform_documents(oppenheimer_csv_docs)


In [143]:
#csv retriever
opp_csv_faiss_store = FAISS.from_documents(chunked_opp_csv_docs, embedder)

opp_csv_faiss_retriever = opp_csv_faiss_store.as_retriever()


In [144]:
# wikipedia retriever

#setup FAISS vector store
opp_wikipedia_faiss_vector_store = FAISS.from_documents(
    chunked_opp_wikipedia_docs, embedder
)

opp_wikipedia_faiss_retriever = opp_wikipedia_faiss_vector_store.as_retriever(search_kwargs={"k": 1})

#setup BM25 Retriever
opp_wikipedia_bm25_retriever = BM25Retriever.from_documents(chunked_opp_wikipedia_docs)

opp_wikipedia_bm25_retriever.k = 1

In [145]:
#ensemble retriever
opp_ensemble_retriever = EnsembleRetriever(
    retrievers=[opp_wikipedia_bm25_retriever, opp_wikipedia_faiss_retriever],
    weights=[0.50, 0.50] # should sum to 1
)

#### Multi-source chain

We're going to allow the LLM to decide which information is most -> least valuable.

The way we'll do this is with LangChain's rather powerful "Expression Language"!

> HINT: You can leverage [this](https://python.langchain.com/docs/use_cases/question_answering/how_to/multiple_retrieval) resource if you get stuck - but experiment with different prompts/formats.

In [124]:
from langchain.prompts import ChatPromptTemplate

system_message = """Use the information from the below two sources to answer any questions.

Source 1: public user reviews about the Oppenheimer movie
<source1>
{source1}
</source1>

Source 2: the wikipedia page for the Oppenheimer movie including the plot summary, cast, and production information
<source2>
{source2}
</source2>
"""

prompt = ChatPromptTemplate.from_messages([("system", system_message), ("human", "{question}")])

In [125]:
oppenheimer_multisource_chain = {
    "source1": (lambda x: x["question"]) | opp_ensemble_retriever,
    "source2": (lambda x: x["question"]) | opp_csv_faiss_retriever,
    "question": lambda x: x["question"],
} | prompt | llm

In [126]:
oppenheimer_multisource_chain.invoke({"question" : "What did people think of the Oppenheimer movie?"})

AIMessage(content="People generally thought highly of the Oppenheimer movie. Critics praised the screenplay, performances of the cast (especially Cillian Murphy and Robert Downey Jr.), and the visuals. It was frequently cited as one of the best films of 2023. On the review aggregator website Rotten Tomatoes, 93% of 451 critics' reviews were positive, with an average rating of 8.6/10. The movie received critical acclaim, with particular praise for the cast performances, screenplay, and visuals. However, some criticism was aimed towards the writing of the female characters. Overall, the movie was considered compelling, engaging, and simple to understand, and it sparked an interest in history.", additional_kwargs={}, example=False)

# Agent Creation

Now we can finally start building our Agent!

The first thing we'll need to do is provide our Agent a Toolbelt. (list of tools). Much like Batman, our LLM-powered Agent can use these tools as it sees fit. 

While the examples we're constructing in this notebook are straightforward for brevity and simplicities sake - there is no limit to what you can build with Agents, as we'll see as we progress through the program.

So, let's begin by setting up our Tools!

You'll notice that we have to set up a function to allow our `OppenheimerInfo` tool to interface with the Agent - this is due to it have a specific required input. Creating custom tools is a pattern that you'll want to grow acustomed to as you use LangChain more and more.

In [151]:
from langchain.agents import Tool

def query_oppenheimer(input):
    """This tool is useful for searching about the Oppenheimer movie on wikipedia and in csv files"""
    return oppenheimer_multisource_chain.invoke({"question" : input})

tools = [
    Tool(
        name = "BarbieTool",
        func=query_oppenheimer,
        description="Useful for answering question about the Barbie movie"
    ),
    Tool(
        name = "OppenheimerTool",
        func = query_oppenheimer,
        description= "Useful for answering question about the Oppenheimer movie"
    ),
]

Now that we've set up our Agents toolbelt, let's set up the LLM that will be powering it!

I would suggest playing around with these prompts - and experiments to find what works best for you.

RELEVANT DOCS:
- [`ZeroShotAgent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.mrkl.base.ZeroShotAgent.html#langchain-agents-mrkl-base-zeroshotagent)

In [152]:
from langchain.agents import ZeroShotAgent, AgentExecutor

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools=tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "agent_scratchpad"]
)

In [153]:
from langchain import LLMChain

llm_chain = LLMChain(llm=llm, prompt=prompt)

All that's left to do now is create our `ZeroShotAgent` and our `AgentExecutor`, which are the "reasoner" and "actor" halfs of the `ReAct` method of Agent implementation.

Read all about the `ReAct` framework [here](https://react-lm.github.io/)

In [154]:
barbenheimer_agent = ZeroShotAgent(
    llm_chain=llm_chain, 
    tools=tools, 
    verbose=True)

barbenheimer_agent_chain = AgentExecutor.from_agent_and_tools(
    agent=barbenheimer_agent, 
    tools=tools, 
    verbose=True)

## Conclusion

All that is left to do now, is feed inputs to your Agent and watch it work!

Remember to use the `{"input" : "YOUR QUERY HERE"}` format when prompting the Agent.

In [155]:
barbenheimer_agent_chain.invoke({"input" : "What did people like about the Barbie movie?"})



> Entering new AgentExecutor chain...
Thought: I don't have any information about the Barbie movie, so I should use the BarbieTool to find out what people liked about it.
Action: BarbieTool
Action Input: "What did people like about the Barbie movie?"
Observation: content='There is no specific information available about what people liked about the Barbie movie in the provided sources.' additional_kwargs={} example=False
Thought:I couldn't find any specific information about what people liked about the Barbie movie. I should try using the OppenheimerTool instead to see if it has any relevant information.
Action: OppenheimerTool
Action Input: "What did people like about the Barbie movie?"
Observation: content='There is no specific information available about what people liked about the Barbie movie in the provided sources.' additional_kwargs={} example=False
Thought:

OutputParserException: Could not parse LLM output: `I couldn't find any specific information about what people liked about the Barbie movie using either tool. I don't have enough information to provide a final answer to the question.`

In [ ]:
barbenheimer_agent_chain.run({"input" : "What did people like about the Oppenheimer movie?"})



> Entering new AgentExecutor chain...
Thought: I don't have any information about the Oppenheimer movie, so I should use the OppenheimerTool to find out what people liked about it.
Action: OppenheimerTool
Action Input: "What did people like about the Oppenheimer movie?"
Observation: content='People liked the screenplay, the performances of the cast (particularly those of Murphy and Downey), and the visuals of the Oppenheimer movie. They also praised its accuracy and the way it made the complicated story of J. Robert Oppenheimer compelling and easy to understand. The film received critical acclaim for its cast performances, screenplay, and visuals.' additional_kwargs={} example=False
Thought:I now know what people liked about the Oppenheimer movie.
Final Answer: People liked the screenplay, the performances of the cast (particularly those of Murphy and Downey), and the visuals of the Oppenheimer movie. They also praised its accuracy and the way it made the complicated story of J. Robe

'People liked the screenplay, the performances of the cast (particularly those of Murphy and Downey), and the visuals of the Oppenheimer movie. They also praised its accuracy and the way it made the complicated story of J. Robert Oppenheimer compelling and easy to understand. The film received critical acclaim for its cast performances, screenplay, and visuals.'

In [ ]:
barbenheimer_agent_chain.run({"input" : "Did the movies Barbie and Oppenheimer share similar themes or ideas?"})



> Entering new AgentExecutor chain...
Thought: I need to compare the themes or ideas of the Barbie and Oppenheimer movies.
Action: BarbieTool
Action Input: Compare themes or ideas of Barbie movie

> Entering new AgentExecutor chain...
Based on the available information, the Barbie movie directed by Greta Gerwig explores themes of self-discovery, existential crisis, and challenging societal expectations. The film follows Barbie and Ken on a journey of self-discovery, suggesting that the characters are questioning their purpose and searching for meaning in their lives. This theme of self-discovery implies a deeper exploration of identity and personal growth.

Additionally, the movie seems to challenge societal expectations and stereotypes associated with Barbie. It presents Barbie as more than just a toy or cultural relic, but as a feminist statement. The film aims to reimagine and redefine Barbie, portraying her in a new and empowering light. This suggests a theme of breaking free fro

'The Barbie movie explores themes of self-discovery, challenging societal expectations, and the importance of relationships and community. It aims to present Barbie in a new and empowering way, promoting individuality and breaking free from traditional gender roles. On the other hand, the Oppenheimer movie explores themes of ethical dilemmas, personal conflict, McCarthyism and paranoia, trust and betrayal, the power of scientific discoveries, and the human cost of war.'

## Next Steps

It's time to build a Chainlit (or Gradio) application and host it on Hugging Face Spaces! :ship: